# Importacion de modulos

In [1]:
#Importacion de modulos requeridos para el tratamiento de la data y los procedimientos
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime


In [210]:
# importamos modeulos requeridos para trabajar con google colab
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Se declara la funcion que sirve para des-anidar los diccionarios contenidos en las columnas

In [1]:
# Se crea una funcion para desempaquetar los diccionarion que estan contenidos dentro de los campos de los dataframes
def unpacked(dataframe:pd.DataFrame, # dataframe que contiene el campo a transformar
             col_dict:str, # nombre de la columna que contiene el diccionario a transformar
             campo_1:str, # nombre del primer nuevo campo
             campo_2:str, # nombre del segundo nuevo campo
             key_1:str, # nombre de la key 1 a extraer del diccionario
             key_2:str): # nombre de la key 2 a extraer del diccionario

  # reseteamos el indice del dataframe para que quede nuevamente en orden numero  y no presente problemas al evaluar
  dataframe.reset_index(inplace=True)
  # creo un nuevo dataframe al cual agregare las nuevas filas para empezar a des-anidar los diccionarios
  aux_dat = dataframe.drop(dataframe.index)
  aux_dat[[campo_1, campo_2]] = 0

  lista_columns = aux_dat.columns.values # se crea una lista con los nombres de las columnas del dataframe auxiliar
  ti = datetime.now()
  mod = 1000
  for i in range(dataframe.shape[0]): # inicio un ciclo con un iterador del tamaño de la cantidad de registros que tiene inicialmente el dataframe
      if i == mod:
        mod += 1000
        print(i)
      aux = eval(dataframe[col_dict][i]) # desesmpaqueto el string a un objeto dipo lista con la funcion eval
      for j in range(len(aux)): # inicio un ciclo del tamaño de la cantidad de generos con la que cuenta la pelicula

          lista_values = []
          if len(aux) > 0:
            lista_values.append(aux[j][key_1]) # agrego inicialmente a la lista vacia el valor de la key
            lista_values.append(aux[j][key_2]) # agrego inicialmente a la lista vacia el valor de la key
          else:
            lista_values.append(np.nan) # agrego inicialmente a la lista vacia el valor de la key
            lista_values.append(np.nan) # agrego inicialmente a la lista vacia el valor de la key

          i_insert = 0 # inicio un punto para insertar en la respectiva posicion y nombre

          for k in dataframe.columns.values: # recorro la lista de nombres que tiene el dataframe movies

              lista_values.insert(i_insert, dataframe[k][i]) # inserto el valor correspondiente del dataframe, nombre y posicion especifica a la lista de valores
              i_insert += 1 # incremento mi puntero en 1

          serie_aux = dict(zip(lista_columns, lista_values)) # creo un diccionario con el nombre y lista de valores
          aux_dat = aux_dat.append(serie_aux, ignore_index=True) # agrego al dataframe aux_dat la serie recien creada

  # calculo del tiempo transcurrido
  tt_seg = (datetime.now() - ti).seconds

  # tratamiento del dateframe, se elimina la columna index
  aux_dat.drop(columns='index', axis=1, inplace=True)
  return aux_dat, tt_seg

NameError: name 'pd' is not defined

# Cargamos el archivo principal movies.csv para empezar con la transformacion de los datos

In [245]:
data = '/content/drive/Othercomputers/Mi portátil/PI/Dataset/movies_dataset.csv'
movies = pd.read_csv(data, sep=',') # importamos los datos como dataframe
movies # vemos como esta construido el dataframe

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


Inicialmente eliminamos todos los registros duplicados existentes en el dataframe

In [212]:
print('Vemos cuantos duplicados hay en el dataframe antes de eliminar: ', movies.duplicated().sum())
movies.drop_duplicates(inplace=True)
print('Vemos cuantos duplicados hay en el dataframe despues de eliminar: ', movies.duplicated().sum())

Vemos cuantos duplicados hay en el dataframe antes de eliminar:  13
Vemos cuantos duplicados hay en el dataframe despues de eliminar:  0


Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [213]:
print('Shape del data frame antes de eliminar columnas que no se usaran: ', movies.shape)
movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], axis=1, inplace=True)
print('Shape del data frame despues de eliminar columnas que no se usaran: ', movies.shape)

Shape del data frame antes de eliminar columnas que no se usaran:  (45453, 24)
Shape del data frame despues de eliminar columnas que no se usaran:  (45453, 18)


Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [214]:
# imputamos con un 0 los valores nulos del campo revenue
print('Cantidad de valores nulos en el campo revenue antes de imputar: ', movies.revenue[movies.revenue.isna()].shape[0])
movies.revenue[movies.revenue.isna()] = 0
print('Cantidad de valores nulos en el campo revenue despues de imputar: ', movies.revenue[movies.revenue.isna()].shape[0])

Cantidad de valores nulos en el campo revenue antes de imputar:  6
Cantidad de valores nulos en el campo revenue despues de imputar:  0


In [215]:
# imputamos con un 0 los valores nulos del campo budget
print('Cantidad de valores nulos en el campo budget antes de imputar: ', movies.budget[movies.budget.isna()].shape[0])
movies.budget[movies.budget.isna()] = 0
print('Cantidad de valores nulos en el campo revenue despues de imputar: ', movies.budget[movies.budget.isna()].shape[0])

Cantidad de valores nulos en el campo budget antes de imputar:  0
Cantidad de valores nulos en el campo revenue despues de imputar:  0


Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

Convertir el campo budget a numero para poder crear la nueva columna return

In [216]:
# la columna budget tienes algunos registros no numericos, imputamos 0 a estos
movies.budget[movies.budget == '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'] = 0
movies.budget[movies.budget == '/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg'] = 0
movies.budget[movies.budget == '/zaSf5OG7V8X8gqFvly88zDdRm46.jpg'] = 0

In [217]:
# convertimos la columna a dato tipo int64
movies.budget = movies.budget.astype('int64')

In [218]:
# Crear la nueva columna como el resultado de la división, evitando la división por cero
movies['return'] = np.where(movies['budget'] != 0, movies['revenue'] / movies['budget'], 0)

Los valores nulos del campo release date deben eliminarse.

In [219]:
print('Cantidad de filas del dataframe antes de eliminar los valores nulos del campo release_date: ', movies.shape[0])
movies = movies[movies.release_date.notna()]
print('Cantidad de filas del dataframe despues de eliminar los valores nulos del campo release_date: ', movies.shape[0])

Cantidad de filas del dataframe antes de eliminar los valores nulos del campo release_date:  45453
Cantidad de filas del dataframe despues de eliminar los valores nulos del campo release_date:  45366


De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [220]:
# eliminamos los registros cuyo id es incorrecto (3)
# estos valores son corruptos y no dejan modificar el formato a timpo fecha
print('cantidad de registros antes de eliminar los que tienen el id incorrecto: ', movies.shape[0])
movies.drop(movies.id[movies.id == '1997-08-20'].index, inplace=True)
movies.drop(movies.id[movies.id == '2012-09-29'].index, inplace=True)
movies.drop(movies.id[movies.id == '2014-01-01'].index, inplace=True)
print('cantidad de registros despues de eliminar los que tienen el id incorrecto: ', movies.shape[0])

cantidad de registros antes de eliminar los que tienen el id incorrecto:  45366
cantidad de registros despues de eliminar los que tienen el id incorrecto:  45363


In [221]:
# verificamos que no hayan datos numeros y extraños
movies.release_date.sort_values()

34940    1874-12-09
34937    1878-06-14
41602    1883-11-19
34933    1887-08-18
34934    1888-01-01
            ...    
44535    2018-04-04
38130    2018-04-25
30402    2018-11-07
38885    2018-12-31
26559    2020-12-16
Name: release_date, Length: 45363, dtype: object

In [222]:
# convertimos la columna a tipo fecha
movies.release_date = pd.to_datetime(movies.release_date)
movies.release_date.sort_values()

34940   1874-12-09
34937   1878-06-14
41602   1883-11-19
34933   1887-08-18
34934   1888-01-01
           ...    
44535   2018-04-04
38130   2018-04-25
30402   2018-11-07
38885   2018-12-31
26559   2020-12-16
Name: release_date, Length: 45363, dtype: datetime64[ns]

Creamos la columna release_year donde extraerán el año de la fecha de estreno.

In [223]:
# Extraemos el año a partir de la columna release_year y creamos un nuevo campo
movies['release_year'] = movies.release_date.dt.year
movies.release_year

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45460    1991
45462    2011
45463    2003
45464    1917
45465    2017
Name: release_year, Length: 45363, dtype: int64

Se procede a desempaquetar el campo 'genres', si una pelicula tiene por ejemplo 3 generos se hara una fila
por cada genero al que este asociada, es decir, quedaran 3 filas y cada una con un genero independiente.

In [226]:
aux_mov, tiempo_transcurrido = unpacked(movies, 'genres', 'id_genero', 'genero', 'id', 'name' )

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
1782


,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,id_genero,genero
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,16,Animation
1,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,35,Comedy
2,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,10751,Family
3,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,1995,12,Adventure
4,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,1995,14,Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90999,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",30840,en,"Yet another version of the classic epic, with ...",5.683753,"[{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1991-05-13,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Robin Hood,5.7,26.0,0.000000,1991,10749,Romance
91000,NaN,0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,An artist struggles to finish his work while a...,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,...,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,9.0,3.0,0.000000,2011,18,Drama
91001,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,3.8,6.0,0.000000,2003,28,Action
91002,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,3.8,6.0,0.000000,2003,18,Drama


Exportamos el dataframe con y sin la columna genres

In [243]:
# se guarda en csv y se exporta con la columna genres
aux_mov.to_csv('1_ttm_1.csv')
files.download('1_ttm_1.csv')

# se elimina la columna genres, se guarda en csv y se exporta sin la columna genres
aux_mov.drop(columns='genres', inplace=True) # se elimina la columna de generos ya que se hizo la desanidacion
aux_mov.to_csv('1_ttm_0.csv')
files.download('1_ttm_0.csv')

# se muestra el encabezado del nuevo dataframe y se revisa que haya quedado acorde
aux_mov.head(2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,belongs_to_collection,budget,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,id_genero,genero
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,16,Animation
1,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,35,Comedy


# En este momento se procede a hacer el tratamiento de otro diccionario belong_to_collecion






In [299]:
data = '/content/drive/Othercomputers/Mi portátil/PI/Procedimientos/Transformaciones/2/1_ttm_0.csv'
movies = pd.read_csv(data, index_col=0) # cargamos el dataframe
movies.head(2)

,belongs_to_collection,budget,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,id_genero,genero
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,16,Animation
1,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,35,Comedy


Cambiamos el tipo de dato a los campos de tipo numerico de ser posible para optimizar recursos

In [300]:
# convertimos el campo id en tipo int y se verificael numero maximo y se asigna el tipo de dato int32 para optimizar memoria
movies.id = movies.id.astype('int64')
print(movies.id.max())
movies.id = movies.id.astype('int32')
print(movies.id.max())

469172
469172


In [301]:
# convertimos el campo id en tipo int y se verificael numero maximo y se asigna el tipo de dato int32 para optimizar memoria
print(movies.budget.max())
movies.budget = movies.budget.astype('int32')
print(movies.budget.max())

380000000
380000000


In [302]:
# convertimos el campo id en tipo float y se verifica el numero maximo y se asigna el tipo de dato float16 para optimizar memoria
print(movies.vote_average.max())
movies.vote_average = movies.vote_average.astype('float16')
print(movies.vote_average.max())


10.0
10.0


In [303]:
# convertimos el campo id en tipo float y se verifica el numero maximo y se asigna el tipo de dato float32 para optimizar memoria
print(movies.vote_count.max())
movies.vote_count = movies.vote_count.astype('float32')
print(movies.vote_count.max())

14075.0
14075.0


In [304]:
# convertimos el campo id en tipo float y se verificael numero maximo y se asigna el tipo de dato float16 para optimizar memoria
print(movies.runtime .max())
movies.runtime  = movies.runtime .astype('float16')
print(movies.runtime .max())

1256.0
1256.0


In [305]:
# convertimos el campo id en tipo float y se verificael numero maximo y se asigna el tipo de dato float64 para optimizar memoria
print(movies.revenue.max())
movies.revenue = movies.revenue.astype('float64')
print(movies.revenue.max())

2787965087.0
2787965087.0


In [306]:
# convertimos el campo id en tipo float y se verificael numero maximo y se asigna el tipo de dato float32 para optimizar memoria
print(movies['return'].max())
movies['return'] = movies['return'].astype('float32')
print(movies['return'].max())

12396383.0
12396383.0


In [307]:
# vemos la informacion del dataframe
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91004 entries, 0 to 91003
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  11077 non-null  object 
 1   budget                 91004 non-null  int32  
 2   id                     91004 non-null  int32  
 3   original_language      90988 non-null  object 
 4   overview               89935 non-null  object 
 5   popularity             91004 non-null  float64
 6   production_companies   91004 non-null  object 
 7   production_countries   91004 non-null  object 
 8   release_date           91004 non-null  object 
 9   revenue                91004 non-null  float64
 10  runtime                90753 non-null  float16
 11  spoken_languages       91004 non-null  object 
 12  status                 90910 non-null  object 
 13  tagline                46516 non-null  object 
 14  title                  91004 non-null  object 
 15  vo

Haremos el tratamiento al campo belong_to_collection.

El campo belong_to_collection contiene un diccionario con 4 items, de este diccionario solo tomaremos las keys id y name y a partir de estos se hara 2 columnas respectivamente, llamadas id_collecion y nombre_collection.


In [310]:
ti = datetime.now() # iniciamos para contar el tiempo que requiere el proceso
filtro = movies[movies.belongs_to_collection.notna()].index # creo un filtro con los indices de las columnas cuyo valor en el campo belong_to_collection no es nulo
movies[['id_coleccion', 'nombre_coleccion']] = np.nan # creo 2 campos en el dataframe llamadas id_collecion y nombre_collection para traer la informacion del diccionario de belong_to_collection y los inicializo en nan
for i in filtro: # inicio un ciclo recorriendo el vector que contiene los indices de los datos no nulos en belong_to_collection
  # empiezo a llenar los nuevos campos a partir de la informacion extraida del diccionario que contiene belong_to_collection
  aux_belongs_to_collection = eval(movies.belongs_to_collection[i]) # evaluo el registro para convertirlo a diccionario
  movies['id_coleccion'][i] = aux_belongs_to_collection['id'] # inserto al campo id_coleccion el valor de id del diccionario de belong_to_collection
  movies['nombre_coleccion'][i] = aux_belongs_to_collection['name'] # inserto al campo nombre_coleccion el valor de name del diccionario de belong_to_collection

tf = datetime.now()
tt = (tf - ti).seconds # retorno el tiempo transcurrido de este proceso
tt

7

In [313]:
movies.head(2)

,belongs_to_collection,budget,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,tagline,title,vote_average,vote_count,return,release_year,id_genero,genero,id_coleccion,nombre_coleccion
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,NaN,Toy Story,7.699219,5415.0,12.451801,1995,16,Animation,10194.0,Toy Story Collection
1,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,NaN,Toy Story,7.699219,5415.0,12.451801,1995,35,Comedy,10194.0,Toy Story Collection


In [314]:
# se guarda en csv y se exporta con la columna genres
csv_0 = '2_ttm_0.csv'
csv_1 = '2_ttm_1.csv'
movies.to_csv(csv_1)
files.download(csv_1)

# se elimina la columna belongs_to_collection, se guarda en csv y se exporta sin la columna genres
movies.drop(columns='belongs_to_collection', inplace=True) # se elimina la columna de generos ya que se hizo la desanidacion
movies.to_csv(csv_0)
files.download(csv_0)

# se muestra el encabezado del nuevo dataframe y se revisa que haya quedado acorde
movies#.head(2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,budget,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,...,tagline,title,vote_average,vote_count,return,release_year,id_genero,genero,id_coleccion,nombre_coleccion
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,...,NaN,Toy Story,7.699219,5415.0,12.451801,1995,16,Animation,10194.0,Toy Story Collection
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,...,NaN,Toy Story,7.699219,5415.0,12.451801,1995,35,Comedy,10194.0,Toy Story Collection


In [315]:
# vemos la conformacion de columnas del dataframe
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91004 entries, 0 to 91003
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                91004 non-null  int32  
 1   id                    91004 non-null  int32  
 2   original_language     90988 non-null  object 
 3   overview              89935 non-null  object 
 4   popularity            91004 non-null  float64
 5   production_companies  91004 non-null  object 
 6   production_countries  91004 non-null  object 
 7   release_date          91004 non-null  object 
 8   revenue               91004 non-null  float64
 9   runtime               90753 non-null  float16
 10  spoken_languages      91004 non-null  object 
 11  status                90910 non-null  object 
 12  tagline               46516 non-null  object 
 13  title                 91004 non-null  object 
 14  vote_average          91004 non-null  float16
 15  vote_count         

# Se procede a hacer la transformacion de los datos de la columna production_companies

In [331]:
data = '/content/drive/Othercomputers/Mi portátil/PI/Procedimientos/Transformaciones/2/2_ttm_0.csv'
movies = pd.read_csv(data, index_col=0) # cargamos el dataframe
movies.head(2)

,budget,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,...,tagline,title,vote_average,vote_count,return,release_year,id_genero,genero,id_coleccion,nombre_coleccion
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,...,NaN,Toy Story,7.7,5415.0,12.451801,1995,16,Animation,10194.0,Toy Story Collection
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,...,NaN,Toy Story,7.7,5415.0,12.451801,1995,35,Comedy,10194.0,Toy Story Collection


In [332]:
print('cantidad de duplicados antes de eliminarlos: ',movies.duplicated().sum())
movies.drop_duplicates(inplace=True)
print('cantidad de duplicados despues de eliminarlos: ',movies.duplicated().sum())

cantidad de duplicados antes de eliminarlos:  8
cantidad de duplicados despues de eliminarlos:  0


In [335]:
# miramos el contenido de los diccionario de production_companies para saber como desempaquetar
movies.production_companies[3]

"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]"

In [ ]:
aux_mov, tiempo_transcurrido = unpacked(movies, 'production_campanies', 'id_companie', 'companie', 'id', 'name' )

In [ ]:
aux_mov.info()

In [341]:
# se guarda en csv y se exporta con la columna genres
csv_0 = '3_ttm_0.csv'
csv_1 = '3_ttm_1.csv'
aux_mov.to_csv(csv_1)
files.download(csv_1)

# se elimina la columna belongs_to_collection, se guarda en csv y se exporta sin la columna genres
aux_mov.drop(columns='production_companies', inplace=True) # se elimina la columna de generos ya que se hizo la desanidacion
aux_mov.to_csv(csv_0)
files.download(csv_0)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Se procede a hacer el tratramiento del diccionario alojado en el campo production_countries

In [371]:
data = '/content/drive/Othercomputers/Mi portátil/PI/Procedimientos/Transformaciones/2/3_ttm_0.csv'
movies = pd.read_csv(data, index_col=0) # cargamos el dataframe
movies#.head(2)

,budget,id,original_language,overview,popularity,production_countries,release_date,revenue,runtime,spoken_languages,...,vote_average,vote_count,return,release_year,id_genero,genero,id_coleccion,nombre_coleccion,id_companie,companie
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,7.7,5415.0,12.451801,1995,16,Animation,10194.0,Toy Story Collection,3,Pixar Animation Studios
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,7.7,5415.0,12.451801,1995,35,Comedy,10194.0,Toy Story Collection,3,Pixar Animation Studios
2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,7.7,5415.0,12.451801,1995,10751,Family,10194.0,Toy Story Collection,3,Pixar Animation Studios
3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,6.9,2413.0,4.043035,1995,12,Adventure,NaN,NaN,559,TriStar Pictures
4,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,6.9,2413.0,4.043035,1995,12,Adventure,NaN,NaN,2550,Teitler Film
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157671,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1991-05-13,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,5.7,26.0,0.000000,1991,10749,Romance,NaN,NaN,38978,CanWest Global Communications
157672,0,111109,tl,An artist struggles to finish his work while a...,0.178241,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",...,9.0,3.0,0.000000,2011,18,Drama,NaN,NaN,19653,Sine Olivia
157673,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,3.8,6.0,0.000000,2003,28,Action,NaN,NaN,6165,American World Pictures
157674,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,3.8,6.0,0.000000,2003,18,Drama,NaN,NaN,6165,American World Pictures


In [372]:
print('cantidad de duplicados antes de eliminarlos: ',movies.duplicated().sum())
movies.drop_duplicates(inplace=True)
print('cantidad de duplicados despues de eliminarlos: ',movies.duplicated().sum())

cantidad de duplicados antes de eliminarlos:  0
cantidad de duplicados despues de eliminarlos:  0


In [373]:
# miramos el contenido de los diccionario de production_companies para saber como desempaquetar
movies.production_countries[35]

"[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [ ]:
aux_mov, tiempo_transcurrido = unpacked(movies, 'production_countries', 'iso_3166_1', 'country', 'iso_3166_1', 'name' )

In [375]:
aux_mov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236504 entries, 0 to 236503
Data columns (total 25 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   budget                236504 non-null  int64  
 1   id                    236504 non-null  int64  
 2   original_language     236455 non-null  object 
 3   overview              235160 non-null  object 
 4   popularity            236504 non-null  float64
 5   production_countries  236504 non-null  object 
 6   release_date          236504 non-null  object 
 7   revenue               236504 non-null  float64
 8   runtime               236247 non-null  float64
 9   spoken_languages      236504 non-null  object 
 10  status                236503 non-null  object 
 11  tagline               142296 non-null  object 
 12  title                 236504 non-null  object 
 13  vote_average          236504 non-null  float64
 14  vote_count            236504 non-null  float64
 15  

In [376]:
aux_mov

,budget,id,original_language,overview,popularity,production_countries,release_date,revenue,runtime,spoken_languages,...,return,release_year,id_genero,genero,id_coleccion,nombre_coleccion,id_companie,companie,iso_3166_1,country
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,12.451801,1995,16,Animation,10194.0,Toy Story Collection,3,Pixar Animation Studios,US,United States of America
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,12.451801,1995,35,Comedy,10194.0,Toy Story Collection,3,Pixar Animation Studios,US,United States of America
2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,12.451801,1995,10751,Family,10194.0,Toy Story Collection,3,Pixar Animation Studios,US,United States of America
3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,4.043035,1995,12,Adventure,NaN,NaN,559,TriStar Pictures,US,United States of America
4,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,4.043035,1995,12,Adventure,NaN,NaN,2550,Teitler Film,US,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236499,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1991-05-13,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,0.000000,1991,10749,Romance,NaN,NaN,38978,CanWest Global Communications,US,United States of America
236500,0,111109,tl,An artist struggles to finish his work while a...,0.178241,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",...,0.000000,2011,18,Drama,NaN,NaN,19653,Sine Olivia,PH,Philippines
236501,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,0.000000,2003,28,Action,NaN,NaN,6165,American World Pictures,US,United States of America
236502,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,0.000000,2003,18,Drama,NaN,NaN,6165,American World Pictures,US,United States of America


In [378]:
# se guarda en csv y se exporta con la columna genres
csv_0 = '4_ttm_0.csv'
csv_1 = '4_ttm_1.csv'
aux_mov.to_csv(csv_1)
files.download(csv_1)

# se elimina la columna belongs_to_collection, se guarda en csv y se exporta sin la columna genres
aux_mov.drop(columns='production_countries', inplace=True) # se elimina la columna de generos ya que se hizo la desanidacion
aux_mov.to_csv(csv_0)
files.download(csv_0)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Se procede a hacer el tratamiento de la ultima columna spoken_languajes

In [380]:
data = '/content/drive/Othercomputers/Mi portátil/PI/Procedimientos/Transformaciones/2/4_ttm_0.csv'
movies = pd.read_csv(data, index_col=0) # cargamos el dataframe
movies#.head(2)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,spoken_languages,status,...,return,release_year,id_genero,genero,id_coleccion,nombre_coleccion,id_companie,companie,iso_3166_1,country
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,12.451801,1995,16,Animation,10194.0,Toy Story Collection,3,Pixar Animation Studios,US,United States of America
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,12.451801,1995,35,Comedy,10194.0,Toy Story Collection,3,Pixar Animation Studios,US,United States of America
2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,12.451801,1995,10751,Family,10194.0,Toy Story Collection,3,Pixar Animation Studios,US,United States of America
3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,...,4.043035,1995,12,Adventure,NaN,NaN,559,TriStar Pictures,US,United States of America
4,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,...,4.043035,1995,12,Adventure,NaN,NaN,2550,Teitler Film,US,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236499,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,0.000000,1991,10749,Romance,NaN,NaN,38978,CanWest Global Communications,US,United States of America
236500,0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,...,0.000000,2011,18,Drama,NaN,NaN,19653,Sine Olivia,PH,Philippines
236501,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,0.000000,2003,28,Action,NaN,NaN,6165,American World Pictures,US,United States of America
236502,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,0.000000,2003,18,Drama,NaN,NaN,6165,American World Pictures,US,United States of America


In [381]:
print('cantidad de duplicados antes de eliminarlos: ',movies.duplicated().sum())
movies.drop_duplicates(inplace=True)
print('cantidad de duplicados despues de eliminarlos: ',movies.duplicated().sum())

cantidad de duplicados antes de eliminarlos:  0
cantidad de duplicados despues de eliminarlos:  0


In [383]:
# miramos el contenido de los diccionario de production_companies para saber como desempaquetar
movies.spoken_languages[35]

"[{'iso_639_1': 'fr', 'name': 'Français'}, {'iso_639_1': 'en', 'name': 'English'}]"

In [ ]:
aux_mov, tiempo_transcurrido = unpacked(movies, 'spoken_languages', 'iso_639_1', 'languaje', 'iso_639_1', 'name')

1000
2000
3000
4000
5000


In [ ]:
aux_mov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236504 entries, 0 to 236503
Data columns (total 25 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   budget                236504 non-null  int64  
 1   id                    236504 non-null  int64  
 2   original_language     236455 non-null  object 
 3   overview              235160 non-null  object 
 4   popularity            236504 non-null  float64
 5   production_countries  236504 non-null  object 
 6   release_date          236504 non-null  object 
 7   revenue               236504 non-null  float64
 8   runtime               236247 non-null  float64
 9   spoken_languages      236504 non-null  object 
 10  status                236503 non-null  object 
 11  tagline               142296 non-null  object 
 12  title                 236504 non-null  object 
 13  vote_average          236504 non-null  float64
 14  vote_count            236504 non-null  float64
 15  

In [ ]:
aux_mov

,budget,id,original_language,overview,popularity,production_countries,release_date,revenue,runtime,spoken_languages,...,return,release_year,id_genero,genero,id_coleccion,nombre_coleccion,id_companie,companie,iso_3166_1,country
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,12.451801,1995,16,Animation,10194.0,Toy Story Collection,3,Pixar Animation Studios,US,United States of America
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,12.451801,1995,35,Comedy,10194.0,Toy Story Collection,3,Pixar Animation Studios,US,United States of America
2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,12.451801,1995,10751,Family,10194.0,Toy Story Collection,3,Pixar Animation Studios,US,United States of America
3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,4.043035,1995,12,Adventure,NaN,NaN,559,TriStar Pictures,US,United States of America
4,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,4.043035,1995,12,Adventure,NaN,NaN,2550,Teitler Film,US,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236499,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1991-05-13,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,0.000000,1991,10749,Romance,NaN,NaN,38978,CanWest Global Communications,US,United States of America
236500,0,111109,tl,An artist struggles to finish his work while a...,0.178241,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",...,0.000000,2011,18,Drama,NaN,NaN,19653,Sine Olivia,PH,Philippines
236501,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,0.000000,2003,28,Action,NaN,NaN,6165,American World Pictures,US,United States of America
236502,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,0.000000,2003,18,Drama,NaN,NaN,6165,American World Pictures,US,United States of America


In [ ]:
# se guarda en csv y se exporta con la columna genres
csv_0 = '4_ttm_0.csv'
csv_1 = '4_ttm_1.csv'
aux_mov.to_csv(csv_1)
files.download(csv_1)

# se elimina la columna belongs_to_collection, se guarda en csv y se exporta sin la columna genres
aux_mov.drop(columns='production_countries', inplace=True) # se elimina la columna de generos ya que se hizo la desanidacion
aux_mov.to_csv(csv_0)
files.download(csv_0)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cargamos el dataframe final, contiene todas las columnas con su respectivo tratamiento y con el requerimiento de cada una de ellas

In [5]:
data = 'D:\DataScience\PI\data_usada_resultante\movies_clean.csv'
movies = pd.read_csv(data, index_col=0)
movies

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,id_genero,genero,id_coleccion,nombre_coleccion,id_companie,companie,iso_3166_1,country,iso_639_1,languaje
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,16,Animation,10194.0,Toy Story Collection,3,Pixar Animation Studios,US,United States of America,en,English
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,35,Comedy,10194.0,Toy Story Collection,3,Pixar Animation Studios,US,United States of America,en,English
2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,10751,Family,10194.0,Toy Story Collection,3,Pixar Animation Studios,US,United States of America,en,English
3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,12,Adventure,NaN,NaN,559,TriStar Pictures,US,United States of America,en,English
4,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,12,Adventure,NaN,NaN,559,TriStar Pictures,US,United States of America,fr,Français
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387454,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,NaN,...,10749,Romance,NaN,NaN,38978,CanWest Global Communications,US,United States of America,en,English
387455,0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,NaN,...,18,Drama,NaN,NaN,19653,Sine Olivia,PH,Philippines,tl,NaN
387456,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,...,28,Action,NaN,NaN,6165,American World Pictures,US,United States of America,en,English
387457,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,...,18,Drama,NaN,NaN,6165,American World Pictures,US,United States of America,en,English
